In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
import os
import sys
import time

In [3]:
sys.path.append("/home/caleml/main-pe/")

In [4]:
from data.datasets.h36m import Human36M
from data.utils.data_utils import TEST_MODE, TRAIN_MODE, VALID_MODE
from data.loader import BatchLoader

from experiments.common import exp_init

In [5]:
from model import blocks
from model import layers
from model import losses
from model import config
from model import callbacks
from model.utils import pose_format, log

In [21]:
from model.networks.multi_branch_model import MultiBranchModel
from model.networks.mbm_vgg import MultiBranchVGGModel

In [22]:
from model.networks.mbm_reduced import MultiBranchReduced

In [8]:
from tensorflow.keras.callbacks import TensorBoard

# Dataset

In [9]:
# local loading
h36m_path = '/home/caleml/datasets/h36m'
h36m = Human36M(h36m_path, dataconf=config.human36m_dataconf, poselayout=pose_format.pa17j3d, topology='frames') 

In [ ]:
conf = {
    'exp_type': 'hybrid_vgg_TEST_NB',
    'dim': 3,
    'n_joints': 17,
    'pose_blocks': 2,
    'dataset_name': 'h36m',
    'batch_size': 8,
    'n_epochs': 60
}

In [10]:
conf = {
    'exp_type': 'hybrid_reduced_TEST_NB',
    'dim': 3,
    'n_joints': 17,
    'pose_blocks': 2,
    'dataset_name': 'h36m',
    'batch_size': 8,
    'n_epochs': 60
}

In [11]:
model_folder = exp_init(conf)

Conducting experiment for 60 epochs and 2 blocks in folder /home/caleml/pe_experiments/exp_20190502_1559_hybrid_reduced_TEST_NB__2b_bs8


In [ ]:
# validation dataset
h36m_val = BatchLoader(
    h36m, 
    ['frame'], 
    ['pose_w', 'pose_uvd', 'afmat', 'camera'], 
    VALID_MODE, 
    batch_size=h36m.get_length(VALID_MODE), 
    shuffle=True)

In [ ]:
log.printcn(log.OKBLUE, 'Preloading Human3.6M validation samples...')
[x_val], [pw_val, puvd_val, afmat_val, scam_val] = h36m_val[0]

In [ ]:
eval_callback = callbacks.H36MEvalCallback(conf['pose_blocks'], x_val, pw_val, afmat_val, puvd_val[:,0,2], scam_val, logdir=model_folder)

# Training

In [ ]:
# VGG with action for phony placeholders
data_tr_h36m = BatchLoader(
        h36m, 
        ['frame'], 
        ['action'] * 3 + ['pose'] * conf['pose_blocks'],
        TRAIN_MODE, 
        batch_size=conf['batch_size'],
        shuffle=True)

In [ ]:
# VGG loader with phony keyword
data_tr_h36m = BatchLoader(
        h36m, 
        ['frame'], 
        ['phony'] * 3 + ['pose'] * conf['pose_blocks'],
        TRAIN_MODE, 
        batch_size=conf['batch_size'],
        shuffle=True)

In [ ]:
# VGG loader with specific phony sizes
data_tr_h36m = BatchLoader(
        h36m, 
        ['frame'], 
        ['phony_2_b_256_256_64', 'phony_2_b_128_128_128', 'phony_2_b_64_64_256'] + ['pose'] * conf['pose_blocks'],
        TRAIN_MODE, 
        batch_size=conf['batch_size'],
        shuffle=True)

In [12]:
# classical i_hat + pose format 
data_tr_h36m = BatchLoader(
        h36m, 
        ['frame'], 
        ['frame'] + ['pose'] * conf['pose_blocks'],
        TRAIN_MODE, 
        batch_size=conf['batch_size'],
        shuffle=True)

y_dict ['frame', 'pose', 'pose'], allkeys ['frame', 'frame', 'pose', 'pose']


### Classical multi branch

In [ ]:
model = MultiBranchModel(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
model.build()

In [ ]:
# model.add_callback(eval_callback)

In [ ]:
model.train(data_tr_h36m, steps_per_epoch=len(data_tr_h36m), model_folder=model_folder, n_epochs=conf['n_epochs'], cb_list=[])

In [ ]:
# short test for cb
model.train(data_tr_h36m, steps_per_epoch=10, model_folder=model_folder, n_epochs=conf['n_epochs'])

### VGG multi branch

In [ ]:
model = MultiBranchVGGModel(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
model.build()

In [ ]:
model.train(data_tr_h36m, steps_per_epoch=len(data_tr_h36m), model_folder=model_folder, n_epochs=conf['n_epochs'], cb_list=[])

### Reduced multi branch

In [23]:
model = MultiBranchReduced(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
model.build()

reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
full ResNet18 model summary
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_34 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_110 (Conv2D)             (None, 128, 128, 64) 9472        input_34[0][0]                   
__________________________________________________________________________________________________
batch_normalization_113 (BatchN (None, 128, 128, 64) 256         conv2d_110[0][0]                 
__________________________________________________________________________________________________
activation_107 (Activation)     (None, 128, 128, 64) 0  

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_34 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_110 (Conv2D)             (None, 128, 128, 64) 9472        input_34[0][0]                   
__________________________________________________________________________________________________
batch_normalization_113 (BatchN (None, 128, 128, 64) 256         conv2d_110[0][0]                 
__________________________________________________________________________________________________
activation_107 (Activation)     (None, 128, 128, 64) 0           batch_normalization_113[0][0]    
__________________________________________________________________________________________________
max_poolin

pose shape (?, 17, 3), vis shape (?, 17, 1), concat shape (?, 17, 4)
pose shape (?, 17, 3), vis shape (?, 17, 1), concat shape (?, 17, 4)
Last H shape Tensor("fReMap2_2/batch_normalization_150/cond/Merge:0", shape=(?, 32, 32, 576), dtype=float32)
pose model summary
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_35 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
Stem (Model)                    (None, 32, 32, 576)  1039488     input_35[0][0]                   
__________________________________________________________________________________________________
rBlock1 (Model)                 (None, 32, 32, 576)  1312128     Stem[1][0]                       
_________________________________________

Build E_a 2.1046242713928223, build E_p 34.71486210823059, decoder D 0.7907085418701172
Input shape (?, 256, 256, 3)
Shape z_a (?, 16, 16, 128), shape z_p (?, 16, 16, 128)
Outputs shape [(None, 256, 256, 3), (None, 17, 4), (None, 17, 4)]
rec y_pred shape (?, 256, 256, 3)
Final model summary
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image_input (InputLayer)        (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
appearance_model (Model)        (None, 16, 16, 128)  690368      image_input[0][0]                
__________________________________________________________________________________________________
pose_model (Model)              [(None, 17, 4), (Non 5103826     image_input[0][0]                
_______________

In [ ]:
cb_list = []

In [ ]:
logs_folder = os.environ['HOME'] + '/pe_experiments/tensorboard/' + model_folder.split('/')[-1]
print('Tensorboard log folder %s' % logs_folder)
tensorboard = TensorBoard(log_dir=os.path.join(logs_folder, 'tensorboard'))

In [ ]:
eval_callback = callbacks.H36MEvalCallback(conf['pose_blocks'], x_val, pw_val, afmat_val, puvd_val[:,0,2], scam_val, pose_only=False, logdir=model_folder)

In [ ]:
cb_list.append(tensorboard)
cb_list.append(eval_callback)
# cb_list.append(LearningRateScheduler(lr_scheduler))
cb_list.append(callbacks.SaveModel(model_folder))

In [ ]:
# train with CB
model.train(data_tr_h36m, steps_per_epoch=10, model_folder=model_folder, n_epochs=conf['n_epochs'], cb_list=cb_list)

In [24]:
# train without CB
model.train(data_tr_h36m, steps_per_epoch=10, model_folder=model_folder, n_epochs=conf['n_epochs'], cb_list=[])

Training with 0 callbacks
Epoch 1/60
10/10 [==============================] - 24s 2s/step - loss: 25.7796 - decoder_loss: 17.5956 - pose_model_loss: 4.1917
Epoch 2/60
10/10 [==============================] - 1s 138ms/step - loss: 14.5364 - decoder_loss: 9.1597 - pose_model_loss: 2.6604
Epoch 3/60
10/10 [==============================] - 1s 136ms/step - loss: 12.9694 - decoder_loss: 8.2089 - pose_model_loss: 2.2836
Epoch 4/60
10/10 [==============================] - 1s 139ms/step - loss: 12.5956 - decoder_loss: 7.7281 - pose_model_loss: 2.3198
Epoch 5/60
10/10 [==============================] - 1s 141ms/step - loss: 12.7248 - decoder_loss: 7.8585 - pose_model_loss: 2.3294
Epoch 6/60
10/10 [==============================] - 1s 132ms/step - loss: 11.2634 - decoder_loss: 6.7069 - pose_model_loss: 2.1887
Epoch 7/60
10/10 [==============================] - 1s 136ms/step - loss: 12.0556 - decoder_loss: 7.4209 - pose_model_loss: 2.2384
Epoch 8/60
10/10 [==============================] - 2s 233

KeyboardInterrupt: 

# VGG debug stuff

In [ ]:
from tensorflow.keras.applications import VGG16

In [ ]:
vgg_model = VGG16(include_top=False, weights='imagenet', input_shape=(256, 256, 3))

In [ ]:
vgg_model.summary()

In [ ]:
output_layers = [1,3,4,6,7]
for i in output_layers:
    print(vgg_model.layers[i].name)

In [ ]:
# conv1_1,conv2_1,conv3_1,pool1,pool2
# from https://discuss.pytorch.org/t/how-to-use-vgg-19-network-to-estimate-perceptual-loss/9981

In [ ]:
a = 'phony_2_b_256_256_64'
batch_size = 24
b = [int(elt) if elt.lower() != 'b' else batch_size for elt in a.split('_')[1:]]

In [ ]:
b

In [29]:
def foo(a, b, c):
    
    def loss(y_true, y_pred):
        return y_true + y_pred + a + b + c
    
    return loss
    

In [30]:
loss = foo(1, 2, 3)

In [32]:
print(type(loss))
loss(10, 10)

<class 'function'>


26

## Dataset debug

In [48]:
import scipy.io as sio
annot_file = os.path.join(h36m_path, 'annotations.mat')
mat = sio.loadmat(annot_file, struct_as_record=False, squeeze_me=True)

In [34]:
sequences = [mat['sequences_te'], mat['sequences_tr'], mat['sequences_val']]
action_labels = mat['action_labels']
joint_labels = mat['joint_labels']

In [37]:
type(joint_labels)
joint_labels.shape

(32,)

In [57]:
# Human 3.6 -> PA17J
joint_labels[[0, 12, 13, 15, 25, 17, 26, 18, 27, 19, 1, 6, 2, 7, 3, 8, 11]]

array(['Hips', 'Spine1', 'Neck', 'Site', 'RightArm', 'LeftArm',
       'RightForeArm', 'LeftForeArm', 'RightHand', 'LeftHand',
       'RightUpLeg', 'LeftUpLeg', 'RightLeg', 'LeftLeg', 'RightFoot',
       'LeftFoot', 'Spine'], dtype=object)

In [51]:
print(type(sequences[1]))
sequences[1][0].shape

<class 'numpy.ndarray'>


AttributeError: 'mat_struct' object has no attribute 'shape'

In [55]:
import numpy as np
a = np.array([[1,2,3], [4,5,6]])
b = np.reshape(a, 6)

print(a)
print(b)

[[1 2 3]
 [4 5 6]]
[1 2 3 4 5 6]
[[1 2 3]
 [4 5 6]]
